# My fuck around and find out playground

In [48]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import os
import pandas as pd
from transformers import BertTokenizer, BertModel
import gensim.downloader as api
import gensim

In [232]:
connections_data_file = "./NYT-Connections-Answers/connections.json"

df = pd.read_json(connections_data_file)['answers']

In [46]:
path = api.load("word2vec-google-news-300", return_path=True)

[--------------------------------------------------] 1.4% 23.4/1662.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[================================================--] 97.8% 1627.0/1662.8MB downloaded


In [52]:
model = gensim.models.keyedvectors.load_word2vec_format(path, binary=True)

In [236]:
import sklearn

In [191]:
for i in df[0]:
    print(i)

{'level': 0, 'group': 'WET WEATHER', 'members': ['HAIL', 'RAIN', 'SLEET', 'SNOW', 'slut', 'slut']}
{'level': 1, 'group': 'NBA TEAMS', 'members': ['BUCKS', 'HEAT', 'JAZZ', 'NETS']}
{'level': 2, 'group': 'KEYBOARD KEYS', 'members': ['OPTION', 'RETURN', 'SHIFT', 'TAB']}
{'level': 3, 'group': 'PALINDROMES', 'members': ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']}


In [192]:
def utility(row):
    out = []
    for level in row:
        out.append(
            [member.lower() for member in level['members']]
        )
    
    return out

In [194]:
utility(df[1])

[['boot', 'loafer', 'pump', 'sneaker'],
 ['foot', 'league', 'mile', 'yard'],
 ['essence', 'people', 'time', 'us'],
 ['are', 'queue', 'sea', 'why']]

In [237]:
df = df.apply(lambda x: utility(x))

In [240]:
flat = [x for row in df[0] for x in row]

In [241]:
flat

['hail',
 'rain',
 'sleet',
 'snow',
 'bucks',
 'heat',
 'jazz',
 'nets',
 'option',
 'return',
 'shift',
 'tab',
 'kayak',
 'level',
 'mom',
 'racecar']

In [205]:
from sklearn.cluster import KMeans

In [225]:
import numpy as np

In [216]:
def get_vector_dict(words: list[str]):
    out = {}
    for word in words:
        vector = model.get_vector(word)
        out[vector] = word
    return out

In [226]:
def get_vectors(words: list[str]):
    lst_of_vecs = []
    for word in words:
        lst_of_vecs.append(model.get_vector(word, norm=True))
    return np.array(lst_of_vecs)

In [261]:
from random import shuffle

In [263]:
shuffle(flat)
kmeans = KMeans(n_clusters=4).fit_predict(get_vectors(flat))

array([1, 0, 2, 3, 1, 2, 3, 0, 3, 1, 2, 3, 2, 1, 1, 2], dtype=int32)